In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 2, 16), datetime.date(2023, 2, 15))

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
#yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-02-16
yesterday: 2023-02-15 00:00:00


In [3]:
yesterday = yesterday.date()
a_year_ago = yesterday - timedelta(days=365)
yesterday, a_year_ago

(datetime.date(2023, 2, 15), datetime.date(2022, 2, 15))

### Restart and Run All Cells

In [4]:
#cols = 'fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','chg_acc':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','%chg_acc':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [5]:
pd.read_sql_query('SELECT COUNT(*) AS records FROM sales', conlite)

,records
0,2091


In [7]:
sqlDel = """DELETE FROM sales"""
rp = conlite.execute(sqlDel)
rp.rowcount

0

In [8]:
sql = """
SELECT name
FROM orders 
ORDER BY name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'ASK', 'BANPU', 'CK', 'CKP', 'DIF', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MC', 'MEGA', 'ORI', 'PTT', 'RCL', 'SAT', 'SINGER', 'TCAP', 'TMT', 'TTB', 'WHAIR'"

In [9]:
len(in_p.split(','))

21

### Get past one year data

In [10]:
sql = """
SELECT name, date, price, qty, maxp, minp 
FROM price 
WHERE date >= '%s' AND date < '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (a_year_ago, today, in_p)
print(sql)


SELECT name, date, price, qty, maxp, minp 
FROM price 
WHERE date >= '2022-02-15' AND date < '2023-02-16' AND name IN ('ASK', 'BANPU', 'CK', 'CKP', 'DIF', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MC', 'MEGA', 'ORI', 'PTT', 'RCL', 'SAT', 'SINGER', 'TCAP', 'TMT', 'TTB', 'WHAIR') 
ORDER BY name, date


In [11]:
df = pd.read_sql(sql, const)
df.sample(5)

,name,date,price,qty,maxp,minp
780,CKP,2022-05-06,5.25,16337534,5.30,5.20
2043,JMT,2022-07-20,70.25,11254153,71.00,68.75
2553,MC,2022-08-26,9.80,560676,9.80,9.65
34,ASK,2022-04-05,45.00,536559,45.75,45.00
3910,SINGER,2022-03-22,49.75,2837419,50.75,49.75


In [12]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df.to_csv(output_file, header=True, index=False)
df.to_csv(osd_file, header=True, index=False)

### Create monitors from orders

In [13]:
sql = """
SELECT name,trade 
FROM orders 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
orders = pd.read_sql(sql, conlite)
orders.set_index(['name'],inplace=True)
orders

,trade
name,
ASK,B
BANPU,B
CK,B
CKP,B
DIF,S
IVL,S
JASIF,S
JMART,B
JMT,B


In [14]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
osd_file = osd_path + file_name

orders.to_csv(data_file, header=None)
orders.to_csv(output_file)
orders.to_csv(osd_file, header=True, index=False)

In [15]:
sql = """
SELECT trade, COUNT(*) AS items 
FROM orders
GROUP BY trade
ORDER BY trade
"""
grp = pd.read_sql(sql, conlite)
grp

,trade,items
0,B,15
1,S,6


### Call ruby ruby\crt-sales.rb

In [16]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Ad hoc'

In [17]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [18]:
!ruby ruby\\crt-sales.rb

ruby/../Data/Yearly-Price-by-Name.csv
name,fm_date,to_date,days,fm_price,to_price,diff,pct,spd,avg,max_price,min_price,trend,nbr,ttl_qty
ASK|2022-02-15|2022-02-15|1|43.5|43.5|0.0|0.0|1|1|44.25|43.0||2|628,767
ASK|2022-02-17|2022-02-17|1|44.75|44.75|0.0|0.0|1|1|45.0|44.0|Upward|3|1,297,147
ASK|2022-02-18|2022-02-18|1|44.5|44.5|0.0|0.0|1|1|44.75|44.0|Downward|4|380,490
ASK|2022-02-21|2022-02-23|3|45.0|46.5|1.5|3.33|7|2|48.0|44.25|Upward|7|5,763,452
ASK|2022-02-24|2022-02-24|1|45.75|45.75|0.0|0.0|1|1|47.0|44.75|Downward|8|1,077,191
ASK|2022-02-25|2022-02-28|2|47.0|49.0|2.0|4.26|9|4|49.0|45.25|Upward|10|3,991,436
ASK|2022-03-01|2022-03-08|6|48.75|44.25|-4.5|-9.23|-17|-3|49.5|42.25|Downward|16|4,888,704
ASK|2022-03-09|2022-03-10|2|46.0|46.0|0.0|0.0|1|0|47.5|43.75|Upward|18|3,008,196
ASK|2022-03-11|2022-03-14|2|45.25|44.5|-0.75|-1.66|-2|-1|46.0|44.0|Downward|20|1,497,100
ASK|2022-03-15|2022-03-17|3|45.0|45.75|0.75|1.67|4|1|46.5|44.25|Upward|23|1,264,457
ASK|2022-03-18|2022-03-25|6|45.5|44.0|

CK|2022-11-28|2022-11-30|3|24.2|24.7|0.5|2.07|6|2|24.7|24.1|Upward|678|17,407,396
CK|2022-12-01|2022-12-02|2|24.4|24.3|-0.1|-0.41|0|0|24.8|24.3|Downward|680|7,765,745
CK|2022-12-06|2022-12-06|1|24.5|24.5|0.0|0.0|1|1|24.5|24.2|Upward|681|3,729,331
CK|2022-12-07|2022-12-07|1|24.0|24.0|0.0|0.0|1|1|24.5|23.9|Downward|682|5,569,251
CK|2022-12-08|2022-12-09|2|24.3|24.5|0.2|0.82|3|1|24.5|24.0|Upward|684|5,888,268
CK|2022-12-13|2022-12-13|1|24.1|24.1|0.0|0.0|1|1|24.6|23.9|Downward|685|6,262,701
CK|2022-12-14|2022-12-16|3|24.2|24.5|0.3|1.24|4|1|24.5|24.1|Upward|688|11,819,757
CK|2022-12-19|2022-12-21|3|24.4|23.2|-1.2|-4.92|-11|-4|24.6|23.2|Downward|691|15,439,525
CK|2022-12-22|2022-12-27|4|23.4|23.9|0.5|2.14|6|1|24.3|23.2|Upward|695|14,226,583
CK|2022-12-28|2022-12-28|1|23.8|23.8|0.0|0.0|1|1|24.0|23.7|Downward|696|3,624,869
CK|2022-12-29|2022-12-30|2|24.2|24.3|0.1|0.41|2|1|24.3|23.8|Upward|698|7,843,794
CK|2023-01-03|2023-01-10|6|23.9|23.5|-0.4|-1.67|-3|-1|24.5|23.2|Downward|704|38,379,065
CK|2

SINGER|2022-11-07|2022-11-07|1|33.75|33.75|0.0|0.0|1|1|36.5|33.0|Downward|4061|20,019,390
SINGER|2022-11-08|2022-11-10|3|34.0|36.5|2.5|7.35|11|3|37.25|33.75|Upward|4064|22,509,688
SINGER|2022-11-11|2022-11-18|6|36.25|31.75|-4.5|-12.41|-17|-3|37.5|31.5|Downward|4070|36,351,473
SINGER|2022-11-21|2022-11-21|1|32.25|32.25|0.0|0.0|1|1|32.75|31.0|Upward|4071|6,873,952
SINGER|2022-11-22|2022-11-24|3|31.75|31.25|-0.5|-1.57|-1|-1|33.0|31.0|Downward|4074|11,584,743
SINGER|2022-11-25|2022-11-25|1|31.5|31.5|0.0|0.0|1|1|31.75|31.0|Upward|4075|2,725,862
SINGER|2022-11-28|2022-11-28|1|30.25|30.25|0.0|0.0|1|1|31.5|30.0|Downward|4076|6,838,780
SINGER|2022-11-29|2022-12-06|6|31.0|32.5|1.5|4.84|7|1|32.75|30.25|Upward|4081|19,159,349
SINGER|2022-12-07|2022-12-13|5|32.25|30.75|-1.5|-4.65|-5|-1|32.75|30.25|Downward|4085|12,941,525
SINGER|2022-12-14|2022-12-14|1|31.0|31.0|0.0|0.0|1|1|31.5|30.75|Upward|4086|2,774,304
SINGER|2022-12-15|2022-12-16|2|28.5|27.5|-1.0|-3.51|-3|-2|30.75|26.25|Downward|4088|27,865,86

In [19]:
os.chdir("C:\\users\\user\\onedrive\\a7\\Ad hoc")
%pwd

'C:\\users\\user\\onedrive\\a7\\Ad hoc'

### call rails runner db/crt_sales.rb

In [20]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-02-15' AND t.status IN ("B","I", "O", "S") 
ORDER BY a.name



In [21]:
df = pd.read_sql(sql, conlite)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('chg_acc = to_price - fm_price',inplace=True)
df['%chg_acc'] = round(df['chg_acc']/df['fm_price']*100,2)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,chg_acc,%chg_acc
0,ASK,2023-02-14,2023-02-15,32.25,30.75,3215182,33.00,30.50,B,SET,-1.50,-4.65
1,BANPU,2023-02-14,2023-02-15,11.00,11.00,219310115,11.30,10.90,B,SET50,0.00,0.00
2,CK,2023-02-14,2023-02-15,22.00,22.00,5278851,22.30,21.60,O,SET100,0.00,0.00
3,CKP,2023-02-08,2023-02-15,4.66,4.28,131875538,4.74,4.24,O,SET100,-0.38,-8.15
4,DIF,2023-02-10,2023-02-15,13.30,13.20,48275264,13.40,13.20,I,SET,-0.10,-0.75
5,IVL,2023-02-15,2023-02-15,40.00,40.00,9498388,40.50,39.50,I,SET50,0.00,0.00
6,JASIF,2023-02-10,2023-02-15,8.20,8.15,23164926,8.25,8.10,I,SET,-0.05,-0.61
7,JMART,2023-02-03,2023-02-15,37.25,28.50,165286876,37.75,28.50,B,SET50,-8.75,-23.49
8,JMT,2023-02-15,2023-02-15,42.75,42.75,75936256,50.50,42.00,B,SET50,0.00,0.00
9,KCE,2023-02-14,2023-02-15,47.00,46.75,32681327,48.75,46.75,B,SET100,-0.25,-0.53
